# Playground notebook

In [1]:
%load_ext autoreload
%autoreload 2

## HuBERT test

In [ ]:
from transformers import AutoProcessor, HubertModel
from datasets import load_dataset

In [ ]:
model_name = "facebook/hubert-large-ls960-ft"

processor = AutoProcessor.from_pretrained(model_name)
model = HubertModel.from_pretrained(model_name)

In [ ]:
ds = load_dataset("C:\\Users\\mj115gl\\work_dir\\thesis\\audio-semantics\\data\\LibriSpeech\\dev-clean")

In [ ]:
input_values = processor(ds["train"][2]["audio"]["array"], return_tensors="pt").input_values  # Batch size 1
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])
transcription

In [ ]:
ds["train"][1]["audio"]["array"].shape

In [ ]:
input_values.shape

In [ ]:
logits.shape

In [ ]:
model

In [ ]:
output = model(input_values)

In [ ]:
output.last_hidden_state.shape

## Baseline W2V

In [ ]:
from gensim.models.word2vec import Word2Vec, LineSentence
# from gensim.test.utils import datapath

In [ ]:
line_fp = open("data/gtbrg_i.txt", "r", encoding="utf-16")
sentences = LineSentence(line_fp)
# line_fp.close()

In [ ]:
line_fp.seek(0)
line_fp.readline()
# line_fp.close()

In [ ]:
vector_size = 100
window = 5
w2v_model_tag = "TEST"
W2V_MODEL_PATH = f"models/w2v_vs{vector_size}_w{window}_{w2v_model_tag}.model"

In [ ]:
w2v_model = Word2Vec(
    sentences,
    window=window,
    vector_size=vector_size,
    min_count=0,
    workers=4,
    epochs=10
)

In [ ]:
w2v_model.save(W2V_MODEL_PATH)

In [ ]:
list(w2v_model.wv.key_to_index.keys())[:15]

In [ ]:
w2v_model.wv.most_similar("man")

In [ ]:
w2v_model.wv.most_similar(positive=["king", "woman"], negative=["man"])

In [ ]:
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
wordsim_scores = []

with open("data/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt") as wordsim_fp:
    for line in wordsim_fp.readlines():
        scores = line.split("\t")
        w1, w2 = scores[0], scores[1]
        gold_score = float(scores[2])
        wordsim_scores.append([w1, w2, gold_score])

In [ ]:
gold_vocab = []
gold_all = []
preds_vocab = []
preds_all = []
tested = 0
oov = 0

for pairs in wordsim_scores:
    w1, w2 = pairs[0].lower(), pairs[1].lower()
    
    try:
        pred = w2v_model.wv.similarity(w1, w2)
        preds_vocab.append(pred)
        gold_vocab.append(pairs[2])
        tested += 1
    
    except KeyError:
        # if w1 not in w2v_model.wv.vocab.keys():
        #     w1_units = sp.EncodeAsPieces(w1)[1:]
        #     w1_vectors = np.array([w2v_model.wv[unit] for unit in w1_units])
        #     w1_vector = w1_vectors.mean(axis=0)
        # else:
        #     w1_vector = w2v_model.wv[w1]
        # if w2 not in w2v_model.wv.vocab.keys():
        #     w2_units = sp.EncodeAsPieces(w2)[1:]
        #     w2_vectors = np.array([w2v_model.wv[unit] for unit in w2_units])
        #     w2_vector = w2_vectors.mean(axis=0)
        # else:
        #     w2_vector = w2v_model.wv[w2]

        # pred = cosine_similarity(w1_vector.reshape(1, -1), w2_vector.reshape(1, -1))
        oov += 1
    
    preds_all.append(pred)
    gold_all.append(pairs[2])
        

print(spearmanr(preds_vocab, gold_vocab)[0], f", tested {tested}/{len(wordsim_scores)} pairs")
print(spearmanr(preds_all, gold_all)[0], f", including OOV")

In [ ]:
simlex_scores = []

with open("data/SimLex-999/SimLex-999.txt") as simlex_fp:
    for line in simlex_fp.readlines()[1:]:
        scores = line.split("\t")
        w1, w2 = scores[0], scores[1]
        gold_score = float(scores[3])
        simlex_scores.append([w1, w2, gold_score])

In [ ]:
gold_vocab = []
gold_all = []
preds_vocab = []
preds_all = []
tested = 0
oov = 0

for pairs in simlex_scores:
    w1, w2 = pairs[0].lower(), pairs[1].lower()
    
    try:
        pred = w2v_model.wv.similarity(w1, w2)
        preds_vocab.append(pred)
        gold_vocab.append(pairs[2])
        tested += 1
    
    except KeyError:
        # if w1 not in w2v_model.wv.vocab.keys():
        #     w1_units = sp.EncodeAsPieces(w1)[1:]
        #     w1_vectors = np.array([w2v_model.wv[unit] for unit in w1_units])
        #     w1_vector = w1_vectors.mean(axis=0)
        # else:
        #     w1_vector = w2v_model.wv[w1]
        # if w2 not in w2v_model.wv.vocab.keys():
        #     w2_units = sp.EncodeAsPieces(w2)[1:]
        #     w2_vectors = np.array([w2v_model.wv[unit] for unit in w2_units])
        #     w2_vector = w2_vectors.mean(axis=0)
        # else:
        #     w2_vector = w2v_model.wv[w2]

        # pred = cosine_similarity(w1_vector.reshape(1, -1), w2_vector.reshape(1, -1))
        oov += 1
    
    preds_all.append(pred)
    gold_all.append(pairs[2])
        

print(spearmanr(preds_vocab, gold_vocab)[0], f", tested {tested}/{len(simlex_scores)} pairs")
print(spearmanr(preds_all, gold_all)[0], f", including OOV")

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
n_clusters = []
words = list(w2v_model.wv.key_to_index.keys())

tqdm_iterator = tqdm(range(0, 99, 1))

for threshold in tqdm_iterator:
    threshold = threshold / 100
    word_to_cluster = dict()  # Stores map from word to cluster
    cluster_to_words = dict()  # Stores map from cluster to words
    cluster_idx = 0  # Counter

    for word in words:
        tqdm_iterator.set_postfix({"Word": word})
        # Check if word has already been clustered
        if word not in word_to_cluster.keys():
            # Create new cluster
            cluster_idx += 1
            # cluster_key = chr(0x0020 + cluster_idx)
            cluster_key = cluster_idx

            # Add new word to cluster
            tqdm_iterator.set_postfix({"status" : "adding new word to cluster"})
            cluster_to_words[cluster_key] = [word]
            word_to_cluster[word] = cluster_key
            
            # Add all similar words
            tqdm_iterator.set_postfix({"status" : "adding all words"})
            for similar_word, score in w2v_model.wv.most_similar(word, topn=50):
                tqdm_iterator.set_postfix({"status" : "checked 50 words"})
                if score > threshold:
                    cluster_to_words[cluster_key].append(similar_word)
                    word_to_cluster[similar_word] = cluster_key
    
    n_clusters.append(len(cluster_to_words))

In [ ]:
len(words)

In [ ]:
plt.plot(n_clusters)

## Convert old

In [ ]:
from string import ascii_letters

In [ ]:
letters = {
    i: key for i, key in enumerate(ascii_letters)
}

In [ ]:
word_to_key = {}
key_to_word = {}

with open("data/quantized/dev-gold.csv", "r") as key_file:
    for line in key_file.readlines()[1:]:
        dataset, key, _, word = line.strip().split(",")
        if word not in word_to_key:
            word_to_key[word] = {
                'librispeech': [],
                'synthetic': []
            }
        word_to_key[word][dataset].append(key)
        key_to_word[key] = word

In [ ]:
utterances = {}

for dataset in ["librispeech", "synthetic"]:
    with open(f"data/quantized/semantic/dev/{dataset}/quantized_outputs.txt", "r") as utterance_file:
        for line in utterance_file.readlines():
            key, seq = line.strip().split("\t")
            utterance = seq.split(",")[1:]

            key = "ls_" + key_to_word[key] if dataset == "librispeech" else "sy_" + key_to_word[key]
            if key not in utterances:
                utterances[key] = []

            utterances[key].append(
                "".join(
                    [letters[int(v)] for i, v in enumerate(utterance)]
                )
            )

In [ ]:
with open("data/level_wise/level0/utterances_original.txt", "w+", encoding="utf-8") as ufp:
    for word in utterances:
        for utterance in utterances[word]:
            ufp.write(word + "\t" + utterance + "\n")

In [ ]:
sim_pairs = []
rel_pairs = []

with open("data/quantized/dev-pairs.csv", "r") as pairs_file:
    for line in pairs_file.readlines()[1:]:
        dataset, _, w1, w2, sim, rel = line.strip().split(",")
        if sim:
            sim_pairs.append((dataset, w1, w2, float(sim)))
        if rel:
            rel_pairs.append((dataset, w1, w2, float(rel)))

with open("data/level_wise/level0/pairs.txt", "w+", encoding="utf-8") as pairs_fp:
    for pair in sim_pairs:
        dataset, w1, w2, score = pair
        pairs_fp.write(
            ("ls_" + w1 if dataset == "librispeech" else "sy_" + w1) + "," +
            ("ls_" + w2 if dataset == "librispeech" else "sy_" + w2) + "," +
            str(score) + "," + "\n"
        )
    for pair in rel_pairs:
        dataset, w1, w2, score = pair
        pairs_fp.write(
            ("ls_" + w1 if dataset == "librispeech" else "sy_" + w1) + "," +
            ("ls_" + w2 if dataset == "librispeech" else "sy_" + w2) + "," +
            "," + str(score) + "\n"
        )

In [ ]:
str("a")

## FastText

In [2]:
import sentencepiece as spm

In [3]:
sp_model = spm.SentencePieceProcessor()

sp_model.Load("models/original_60k_250x1/level1/unigram_vs60000_lw.model")

True

In [4]:
original_corpus = []

with open("data/level_wise/level0/dev_corpus_original.txt", "r", encoding="utf-8") as ocfp:
    for line in ocfp.readlines():
        original_corpus.append(line.strip())

In [5]:
with open("data/original_60k_250x1/level1/dev_corpus_ft.txt", "w+", encoding="utf-8") as ncfp:
    for line in original_corpus:
        pieces = list(
            filter(
                lambda x: x != "▁",
                sp_model.EncodeAsPieces(line)
            )
        )

        units = [piece.replace("▁", "") for piece in pieces]
        
        ncfp.write(" ".join(units) + "\n")

In [ ]:
import fasttext

In [ ]:
ft_model = fasttext.train_unsupervised(
    "data/original_60k_250x1/level1/corpus_ft.txt",
    "cbow",
    dim=250,
    thread=4,
    epoch=7
)

In [ ]:
type(ft_model)

In [ ]:
ft_model.save_model("models/fasttext_cbow")

In [ ]:
# ft_model = fasttext.load_model("models/fasttext_cbow")

In [ ]:
from levelwise_model.test_bench import LSTestBench
from levelwise_model.utterances import WordToUtteranceMapping

In [ ]:
utterance_mapping = WordToUtteranceMapping(map_file="data/level_wise/level0/utterances_original.txt")

In [ ]:
test_bench = LSTestBench(scores_file="data/level_wise/level0/pairs.txt")

In [ ]:
test_bench.ft_score_and_save(ft_model=ft_model, utterances=utterance_mapping, results_file="results/ft_cbow")

## DistilBERT

In [61]:
from sentence_transformers import SentenceTransformer

In [62]:
model = SentenceTransformer('C:\\Users\\mj115gl\\work_dir\\thesis\\audio-semantics\\models\\db_final')

No sentence-transformers model found with name C:\Users\mj115gl\work_dir\thesis\audio-semantics\models\db_final. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\mj115gl\work_dir\thesis\audio-semantics\models\db_final were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
import sentencepiece as spm

In [64]:
sp_model = spm.SentencePieceProcessor()
sp_model.Load("models/original_60k_250x1/level1/unigram_vs60000_lw.model")

True

In [65]:
from levelwise_model.utterances import WordToUtteranceMapping

In [66]:
utterances = WordToUtteranceMapping(utterances="data/level_wise/level0/utterances_original.txt")

In [67]:
updated_utterances = {}

for word in utterances.utterances:
    updated_utterances[word] = []
    for utt in utterances.utterances[word]:
        pieces = list(
            filter(
                lambda x: x != "▁",
                sp_model.EncodeAsPieces(utt)
            )
        )

        units = [piece.replace("▁", "") for piece in pieces]
        updated_utterances[word].append(
            " ".join(units)
        )

In [68]:
updated_utterances = WordToUtteranceMapping(utterances=updated_utterances)

In [69]:
from levelwise_model.test_bench import LSTestBench

In [70]:
test_bench = LSTestBench("data/level_wise/level0/pairs.txt")

In [71]:
def db_word_vec_fn(utt):
    pieces = list(
        filter(
            lambda x: x != "▁",
            sp_model.EncodeAsPieces(utt)
        )
    )

    units = [piece.replace("▁", "") for piece in pieces]
    return model.encode(" ".join(units)).reshape(1, -1)

In [72]:
test_bench.run_suite(
    utterances=updated_utterances,
    word_vec_fn=db_word_vec_fn,
    results_file="results/db-50.txt"
)

{'similarities': {'score': {'librispeech': 8.253180987037307,
   'synthetic': -0.004998110381584656},
  'errors': 0,
  'trials': 1013},
 'xw-abx-test': {'ABX Result': {'librispeech': 0.581140350877193,
   'synthetic': 0.49278846153846156,
   'combined': 0.5240683229813664}},
 'sw-abx-test': {'Same-word ABX Result': {'librispeech': 0.66,
   'synthetic': 0.71,
   'mixed': 0.6457142857142857}}}

In [ ]:
# # Test positive
# ls_sims_pos = []
# sy_sims_pos = []
# tested = 0

# for word in updated_utterances:
#     if len(updated_utterances[word]) > 1:
#         tested += 1
#         utterances_list = [
#             " ".join(
#                 [
#                     piece.replace("▁", "")
#                     for piece in list(
#                         filter(
#                             lambda x: x != "▁",
#                             sp_model.EncodeAsPieces(utterance)
#                         )
#                     )
#                 ]
#             ) for utterance in updated_utterances[word]
#         ]

#         embeddings = model.encode(utterances_list)
#         similarities = cosine_similarity(embeddings)

#         if word.startswith("ls_"):
#             ls_sims_pos.append(np.mean(similarities[np.where(np.tril(similarities, -1))]))
#         if word.startswith("sy_"):
#             sy_sims_pos.append(np.mean(similarities[np.where(np.tril(similarities, -1))]))

# print(f"Avg positive score (LS) = {sum(ls_sims_pos)/len(ls_sims_pos)} ({len(ls_sims_pos)} words).")
# print(f"Avg positive score (SY) = {sum(sy_sims_pos)/len(sy_sims_pos)} ({len(sy_sims_pos)} words).")

In [ ]:
# n_utts = []

# for word in updated_utterances:
#     n_utts.append(len(updated_utterances[word]))

# print(sum(n_utts)/len(n_utts))

In [ ]:
# # Test negative
# ls_sims_neg = []
# sy_sims_neg = []
# tested = 0
# n_negative_samples = 5

# ls_word_list = set([word for word in updated_utterances.keys() if word.startswith("ls_")])
# sy_word_list = set([word for word in updated_utterances.keys() if word.startswith("sy_")])

# for word in updated_utterances:
#     tested += 1

#     negative_samples = []
#     if word.startswith("ls_"):
#         use_list = ls_word_list
#     else:
#         use_list = sy_word_list
#     sample = np.random.choice(list(use_list - {word}), size=(n_negative_samples))
#     for s in sample:
#         negative_samples.append(
#             np.random.choice(updated_utterances[s])
#         )

#     utterances_list = [
#         " ".join(
#             [
#                 piece.replace("▁", "")
#                 for piece in list(
#                     filter(
#                         lambda x: x != "▁",
#                         sp_model.EncodeAsPieces(utterance)
#                     )
#                 )
#             ]
#         ) for utterance in negative_samples
#     ]
    
#     embeddings = model.encode(utterances_list)
#     similarities = cosine_similarity(embeddings)

#     if word.startswith("ls_"):
#         ls_sims_neg.append(np.mean(similarities[np.where(np.tril(similarities, -1))]))
#     if word.startswith("sy_"):
#         sy_sims_neg.append(np.mean(similarities[np.where(np.tril(similarities, -1))]))

# print(f"Avg negative score (LS) = {sum(ls_sims_neg)/len(ls_sims_neg)} ({len(ls_sims_neg)} words).")
# print(f"Avg negative score (SY) = {sum(sy_sims_neg)/len(sy_sims_neg)} ({len(sy_sims_neg)} words).")

In [ ]:
# sum(sy_sims)/len(sy_sims)

### ABX Test

In [57]:
from gensim.models.word2vec import Word2Vec
import numpy as np

In [58]:
w2v_model = Word2Vec.load("models/original_60k_250x1/level1/w2v_vs250_w1_lw.model")

In [59]:
def wv_word_vec_fn(utt):
    if utt in w2v_model.wv.key_to_index.keys():
        return w2v_model.wv[utt].reshape(1, -1)
    else:
        pieces = list(
            filter(
                lambda x: x != "▁",
                sp_model.EncodeAsPieces(utt)
            )
        )

        units = [piece.replace("▁", "") for piece in pieces]

        vectors = np.array([w2v_model.wv[unit] for unit in units])
        return vectors.mean(axis=0).reshape(1, -1)

In [60]:
test_bench.run_suite(
    utterances=utterances,
    word_vec_fn=wv_word_vec_fn,
    results_file="results/w2v.txt"
)

{'similarities': {'score': {'librispeech': 9.01030729942617,
   'synthetic': 1.028987880071217},
  'errors': 0,
  'trials': 1013},
 'xw-abx-test': {'ABX Result': {'librispeech': 0.49780701754385964,
   'synthetic': 0.46033653846153844,
   'combined': 0.47360248447204967}},
 'sw-abx-test': {'Same-word ABX Result': {'librispeech': 0.9057142857142857,
   'synthetic': 0.9571428571428572,
   'mixed': 0.8442857142857143}}}

In [ ]:
from string import ascii_letters

In [ ]:
# preds = 0
# total = 0
# use_noise_for_x = True

# def get_correct_word(pair, word):
#     if pair[0] == word:
#         return pair[1]
#     return pair[1]

# for word_a in updated_utterances:
#     # Sort words by similarity
#     similar_words = sorted(
#         # Get only pairs containing word A
#         filter(
#             lambda x: x[0] == word_a or x[1] == word_a,
#             rel_pairs
#         ),
#         key=lambda x: x[2],
#         reverse=True
#     )

#     similar_words = list(
#         map(
#             lambda x: get_correct_word(x, word_a),
#             similar_words
#         )
#     )

#     for utt_a in updated_utterances[word_a]:
#         word_b = similar_words[0]
#         utt_b = np.random.choice(
#             updated_utterances[word_b]
#         )

#         if not use_noise_for_x:
#             word_x = similar_words[-1]
#             utt_x = np.random.choice(
#                 updated_utterances[word_x]
#             )

#         else:
#             utt_x = np.random.choice(
#                 list(ascii_letters),
#                 len(utt_a),
#                 replace=True
#             )
#             utt_x = "".join(utt_x)

#         v_a = word_vec_fn(utt_a).reshape(1, -1)
#         v_b = word_vec_fn(utt_b).reshape(1, -1)
#         v_x = word_vec_fn(utt_x).reshape(1, -1)

#         if cosine_similarity(v_a, v_b) > cosine_similarity(v_a, v_x):
#             preds += 1
#         total += 1

# print({"ABX Result": preds/total})

In [52]:
# preds = 0
# total = 0
# use_noise_for_x = False

# def get_correct_word(pair, word):
#     if pair[0] == word:
#         return pair[1]
#     return pair[1]

# with open("results/abx_test_db.txt", "w+") as abx_results_file:
#     abx_results_file.write("A,B,X,sim(AB),sim(AX),chosen\n")
#     for word_a in updated_utterances:
#         # Sort words by similarity
#         similar_words = sorted(
#             # Get only pairs containing word A
#             filter(
#                 lambda x: x[0] == word_a or x[1] == word_a,
#                 rel_pairs
#             ),
#             key=lambda x: x[2],
#             reverse=True
#         )

#         similar_words = list(
#             map(
#                 lambda x: get_correct_word(x, word_a),
#                 similar_words
#             )
#         )

#         for utt_a in updated_utterances[word_a]:
#             word_b = similar_words[0]
#             utt_b = np.random.choice(
#                 updated_utterances[word_b]
#             )

#             if not use_noise_for_x:
#                 word_x = similar_words[-1]
#                 utt_x = np.random.choice(
#                     updated_utterances[word_x]
#                 )

#             else:
#                 word_x = "noise"
#                 utt_x = np.random.choice(
#                     list(ascii_letters),
#                     len(utt_a),
#                     replace=True
#                 )
#                 utt_x = "".join(utt_x)

#             if len(set([word_a, word_b, word_x])) == 3:
#                 v_a = word_vec_fn(utt_a).reshape(1, -1)
#                 v_b = word_vec_fn(utt_b).reshape(1, -1)
#                 v_x = word_vec_fn(utt_x).reshape(1, -1)

#                 sim_ab = cosine_similarity(v_a, v_b)
#                 sim_ax = cosine_similarity(v_a, v_x)

#                 chosen = "X"
#                 if sim_ab > sim_ax:
#                     chosen = "B"
#                     preds += 1
#                 total += 1

#                 abx_results_file.write(",".join([word_a, word_b, word_x, str(sim_ab), str(sim_ax), chosen]) + "\n")

# print({"ABX Result": preds/total})

{'ABX Result': 0.5062111801242236}


In [ ]:
# len(set([word_a, word_b, word_x]))

In [ ]:
# sim_pairs = []
# rel_pairs = []

# with open("data/level_wise/level0/pairs.txt", "r") as pairs_file:
#     for line in pairs_file.readlines()[1:]:
#         w1, w2, sim, rel = line.strip().split(",")
#         if sim:
#             sim_pairs.append((w1, w2, float(sim)))
#         if rel:
#             rel_pairs.append((w1, w2, float(rel)))

## Sparse vector baseline

In [7]:
import sentencepiece as spm

In [8]:
sp_model = spm.SentencePieceProcessor()
sp_model.Load("models/original_60k_250x1/level1/unigram_vs60000_lw.model")

True

In [9]:
import numpy as np

In [53]:
def sv_word_vec_fn(word):
    vector = np.zeros(60000)
    tokens = sp_model.EncodeAsIds(word)
    vector[tokens] = 1
    return vector.reshape(1, -1)

In [10]:
from levelwise_model.test_bench import LSTestBench
from levelwise_model.utterances import WordToUtteranceMapping

In [12]:
test_bench = LSTestBench("data/base_data/pairs.txt")
utterances = WordToUtteranceMapping(utterances="data/base_data/utterances_original.txt")

In [56]:
test_bench.run_suite(
    utterances=utterances,
    word_vec_fn=sv_word_vec_fn,
    results_file="results/baseline_sparse_vectors.txt"
)

{'similarities': {'score': {'librispeech': -6.103752173089786,
   'synthetic': 0.2691688253264706},
  'errors': 0,
  'trials': 1013},
 'xw-abx-test': {'ABX Result': {'librispeech': 0.40131578947368424,
   'synthetic': 0.18389423076923078,
   'combined': 0.2608695652173913}},
 'sw-abx-test': {'Same-word ABX Result': {'librispeech': 0.564,
   'synthetic': 0.454,
   'mixed': 0.448}}}

## Generate ABX test pairs

In [1]:
from levelwise_model.utterances import WordToUtteranceMapping

In [2]:
utterances = WordToUtteranceMapping(map_file="data/level_wise/level0/utterances_original.txt")

In [5]:
import numpy as np

In [12]:
a = None

In [13]:
isinstance(a, dict)

False

In [ ]:
all_utterances = []
for word in utterances.utterances:
    for utt in utterances.utterances[word]:
        all_utterances.append(word)

In [ ]:
with open("data/level_wise/level0/abx_sw_original.txt", "w+", encoding="utf-8") as abx_file:
    for word_a in utterances.utterances:
        for utt_a in utterances.utterances[word_a]:
            

## Human ABX

In [5]:
from levelwise_model.utterances import WordToUtteranceMapping

In [6]:
utterances = WordToUtteranceMapping("data/base_data/utterances_original.txt")

In [3]:
import numpy as np

In [6]:
test_sets = {
    'librispeech': {
        'preds': 0,
        'total': 0,
        'dataset': utterances.ls_utterances
    },
    'synthetic': {
        'preds': 0,
        'total': 0,
        'dataset': utterances.sy_utterances
    },
    'mixed': {
        'preds': 0,
        'total': 0,
        'dataset': utterances.mixed_utterances
    },
}

preds = 0
total = 0


with open("test_file.txt", "w+") as test_file:
    test_file.write("Test set\tIndex\tA\tB\tX\n")
    test_file.write("Test set\tIndex\tWord\tUtterance\n")
    with open("solutions.txt", "w+") as solutions_file:
        for test_set in test_sets:
            ds = test_sets[test_set]['dataset']
            for index in range(20):
                # Select two random words for A and X
                flag = True
                while flag:
                    word_a, word_x = np.random.choice(
                        list(ds.keys()),
                        size=2,
                        replace=False
                    )
                    utt_a, utt_b = np.random.choice(
                        ds[word_a],
                        size=2,
                        replace=False
                    )

                    utt_x = np.random.choice(
                        ds[word_x]
                    )
                    if len(set([utt_a, utt_b, utt_x])) == 3:
                        flag = False

                test_file.write(test_set + "\t" + str(index) + "\t")
                solutions_file.write(test_set + "\t" + str(index) + "\t")
                test_file.write(utt_a + "\t")
                if np.random.random() < 0.5:
                    test_file.write(utt_b + "\t" + utt_x + "\n")
                    solutions_file.write("B\t" + utt_b + "\n")
                else:
                    test_file.write(utt_x + "\t" + utt_b + "\n")
                    solutions_file.write("X\t" + utt_x + "\n")

## Test different W2V methods

In [15]:
from gensim.models.word2vec import Word2Vec
import sentencepiece as spm

In [3]:
import numpy as np

In [16]:
sp_model = spm.SentencePieceProcessor()
sp_model.Load("models/original_60k_250x1/level1/unigram_vs60000_lw.model")
w2v_model = Word2Vec.load("models/original_60k_250x1/level1/w2v_vs250_w1_lw.model")

In [20]:
def weighted_avg_word_vec_fn(word):
    if word in w2v_model.wv.key_to_index.keys():
        return w2v_model.wv[word].reshape(1, -1)
    else:
        pieces = list(
            filter(
                lambda x: x != "▁",
                sp_model.EncodeAsPieces(word)
            )
        )

        units = [piece.replace("▁", "") for piece in pieces]
        weights = [len(unit) for unit in units]

        vectors = np.array([w2v_model.wv[unit] for unit in units])
        return np.average(vectors, axis=0, weights=weights).reshape(1, -1)

In [21]:
utterances

In [22]:
test_bench.run_suite(
    utterances=utterances,
    word_vec_fn=weighted_avg_word_vec_fn,
    results_file="results/weighted_w2v.txt"
)

{'similarities': {'score': {'librispeech': 6.347491997101285,
   'synthetic': -0.6719883444536395},
  'errors': 0,
  'trials': 1013},
 'xw-abx-test': {'ABX Result': {'librispeech': 0.5175438596491229,
   'synthetic': 0.47475961538461536,
   'combined': 0.48990683229813664}},
 'sw-abx-test': {'Same-word ABX Result': {'librispeech': 0.9442857142857143,
   'synthetic': 0.9671428571428572,
   'mixed': 0.8885714285714286}}}

### tf-idf

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [100]:
vocab = []
with open("models/original_60k_250x1/level1/unigram_vs60000_lw.vocab", "r") as vocab_file:
    for line in vocab_file.readlines():
        vocab.append(line.strip().split("\t")[0])
print(len(vocab))
vocab = list(set(vocab))
print(len(vocab))

60000
59580


In [101]:
vectorizer = TfidfVectorizer(input='content', lowercase=False, vocabulary=vocab)

In [102]:
corpus = []
with open("data/base_data/corpus_ft.txt", "r") as corpus_file:
    for line in corpus_file.readlines():
        corpus.append(line.strip())

In [103]:
vectorizer.fit(corpus)

TfidfVectorizer(lowercase=False,
                vocabulary=['JJJJJJJJJJxJJxxx', 'aaazO', 'TTTTTToobbbbIIII',
                            'KKKNNNNNNNNGGo', 'oVVVVpkP', 'HHHHHklllllTT',
                            'yyR', 'h', 'LLLLswww', 'mmmnnnnCCC', 'yyMMMMnn',
                            'LssssuugF', 'KKKKKKOF', 'VVVVyppjj',
                            'QQLLLssssswww', 'ooVVVyypppjj', 'ddddoVVV',
                            'mmmmmmmnFmmmnn', 'ffllllllT', 'obbbbbIIIy',
                            'eeeeeeeeeeeeenFt', 'LLLLssssugg', 'XXXXXF',
                            'yyyypfPPP', 'pkP', 'EEEEEoVVVy',
                            'nFnmnmmmmmmmmmmm', 'rKKKWWWWWW', 'obbbIIpkkQQQ',
                            'GGookkkk', ...])

In [104]:
def tf_idf_word_vec_fn(word):
    if word in w2v_model.wv.key_to_index.keys():
        return w2v_model.wv[word].reshape(1, -1)
    else:
        pieces = list(
            filter(
                lambda x: x != "▁",
                sp_model.EncodeAsPieces(word)
            )
        )

        units = [piece.replace("▁", "") for piece in pieces]
        vectorised = vectorizer.transform([" ".join(units)])
        weights = vectorised.data
        print(weights)
        print(units)

        vectors = np.array([w2v_model.wv[unit] for unit in units])
        return np.average(vectors, axis=0, weights=weights).reshape(1, -1)

In [105]:
test_bench.run_suite(
    utterances=utterances,
    word_vec_fn=tf_idf_word_vec_fn,
    results_file="results/tfidf_w2v.txt"
)

[0.51552197 0.45878498 0.499019   0.33485971 0.40323995]
['UQQ', 'oVVVyDDDDiiii', 'SSTTTzzzzzz', 'oIIII', 'yyyppj']
[0.51552197 0.45878498 0.499019   0.33485971 0.40323995]
['UQQ', 'oVVVyDDDDiiii', 'SSTTTzzzzzz', 'oIIII', 'yyyppj']
[0.41982555 0.4923799  0.22079995 0.44145863 0.35089233 0.46318978]
['kB', 'o', 'VVVyyDDDDDiii', 'SSSTTzzzzzzz', 'IIIIyy', 'yyyyyyyypppjjjj', 'MMMMMM']
Length of weights not compatible with specified axis.


ValueError: x and y must have length at least 2.

In [41]:
test_out = sp_model.EncodeAsPieces("UUUQobbbbbIpkkQQrrAAffffQQQoVVVppppjjrrrKKKKKOOOOOOOOOOMMMMMM")
test_out

['▁',
 'UUUQ',
 'obbbbbIpkk',
 'QQ',
 'rrAAffff',
 'QQQ',
 'oVVVppppjj',
 'rrrKKKKK',
 'OOOOOOOOOOMMMMMM']

In [88]:
y = vectorizer.transform([" ".join(test_out)])

In [89]:
y.data

array([0.4176033 , 0.48186836, 0.41833208, 0.34615082, 0.31373919,
       0.14351974, 0.12405802, 0.40517607])

In [65]:
vectorizer.vocabulary_['UUUQ']

22945